# Proyecto 12

## Descripción del proyecto

El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

In [1]:
# Se importan las librerías necesarias
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

Importamos el dataset `/datasets/car_data.csv` y utilizamos los métodos `info()`, `sample()` y `describe()` para conocer un poco sobre los datos que contiene el dataset

In [2]:
# Importamos el dataset
df = pd.read_csv('/datasets/car_data.csv')

In [3]:
# método info y sample
df.info()
print()
display(df.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
15008,12/03/2016 01:56,1200,bus,2003,manual,91,other,150000,1,gasoline,mercedes_benz,no,12/03/2016 00:00,0,23758,16/03/2016 05:45
214977,02/04/2016 12:46,800,sedan,1990,manual,120,other,150000,4,petrol,volvo,no,02/04/2016 00:00,0,81379,06/04/2016 10:45
246030,18/03/2016 22:49,19200,convertible,2007,manual,272,3er,90000,5,petrol,bmw,no,18/03/2016 00:00,0,73312,06/04/2016 02:16
253464,26/03/2016 17:43,1750,NaN,2000,NaN,0,a6,150000,0,NaN,audi,NaN,26/03/2016 00:00,0,58540,28/03/2016 06:17
63515,20/03/2016 11:50,8499,NaN,2017,auto,224,phaeton,150000,4,gasoline,volkswagen,no,20/03/2016 00:00,0,12103,06/04/2016 10:16


In [4]:
# Se utiliza el método describe()
print(df.describe())

               Price  RegistrationYear          Power        Mileage  \
count  354369.000000     354369.000000  354369.000000  354369.000000   
mean     4416.656776       2004.234448     110.094337  128211.172535   
std      4514.158514         90.227958     189.850405   37905.341530   
min         0.000000       1000.000000       0.000000    5000.000000   
25%      1050.000000       1999.000000      69.000000  125000.000000   
50%      2700.000000       2003.000000     105.000000  150000.000000   
75%      6400.000000       2008.000000     143.000000  150000.000000   
max     20000.000000       9999.000000   20000.000000  150000.000000   

       RegistrationMonth  NumberOfPictures     PostalCode  
count      354369.000000          354369.0  354369.000000  
mean            5.714645               0.0   50508.689087  
std             3.726421               0.0   25783.096248  
min             0.000000               0.0    1067.000000  
25%             3.000000               0.0   30165.

Se van a buscar valores duplicados en el dataset

In [5]:
display(df.duplicated().sum())

262

***Conclusión***

* Podemos observar que hay 5 columnas con valores ausentes. Por conveniencia y por tiempo, los valores ausentes serán sustituidos por la palabra "None" para indicar que es un valor "perdido".
* Podemos ver que hay 262 valores duplicados.
* Por buena práctica, el tipo de datos de las columnas `DateCrawled	`, `DateCreated` y `LastSeen` será cambiado por un tipo de datos `date`.
* Los nombres de las columnas respetan el estilo CamelCase y por conveniencia será dejado este estilo en las columnas, si los nombres no tuvieran un estilo definido serían cambiados al estilo snake_case.

**Valores ausentes**

Como ya se mencionó, los valores ausentes serán reemplazados por la palabra "None", esto se debe a que uno de los modelos que usaremos es CatBoos y esto no resulta un problema para dicha librería ya que dichos valores son categrías separadas.

In [6]:
# Se cambia los NaN por la palabra "None"
df = df.fillna('None')

**Valores duplicados**

Tenemos 262 filas que son completamente iguales. Estas serán removidas para no tener valores repetidos.

In [7]:
# Se hace uso de drop_duplicates()
df = df.drop_duplicates().reset_index(drop=True)

**Cambiar el tipo de datos**

Para un análisis exploratorio de datos, es buena práctica cambiar el tipo de datos de las columnas `DateCrawled	`, `DateCreated` y `LastSeen`. Están son columnas que contitnen las fechas de cuando se descargó el perfil de la base de datos, de creación del perfil y de la última vez que el usuario estuvo activo.

Sin embargo, para nuestro modelo de predicción no son necesarios estos datos porque lo que define el valor de un carro son características diferentes como el modelo, el tipo de caja de cambio, el kilometraje, etcétera. Por lo que se decide que estas columnas serán eliminadas para el entrenamiento de los medelos.

In [8]:
# Se borran las colunmnas DateCrawled, DateCreated y LastSeen.
df = df.drop(['DateCrawled', 'DateCreated', 'LastSeen'], axis=1)

In [9]:
# Verificación de los cambios realizados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354107 entries, 0 to 354106
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Price              354107 non-null  int64 
 1   VehicleType        354107 non-null  object
 2   RegistrationYear   354107 non-null  int64 
 3   Gearbox            354107 non-null  object
 4   Power              354107 non-null  int64 
 5   Model              354107 non-null  object
 6   Mileage            354107 non-null  int64 
 7   RegistrationMonth  354107 non-null  int64 
 8   FuelType           354107 non-null  object
 9   Brand              354107 non-null  object
 10  NotRepaired        354107 non-null  object
 11  NumberOfPictures   354107 non-null  int64 
 12  PostalCode         354107 non-null  int64 
dtypes: int64(7), object(6)
memory usage: 35.1+ MB


### Codificación OHE

Un modelo que será utilizado será LightGBM, este requiere un tipo de codificación OHE para una tarea de regresión. Además para realizar pruebas de cordura se utilizará el bosque aleatorio de regresión y para esto se ocupará igual la codificación OHE.

In [10]:
# Codificación OHE
df_ohe = pd.get_dummies(df, drop_first=True)

# comprobar la codificación
print(df_ohe.shape)

(354107, 315)


## Entrenamiento del modelo 

### División del dataset

Creación de dos funciones, una función para dividir los dataset en features y target. Para esta tarea nuestro objetivo es el precio del vehículo, es decir, la columna `Price`. 

Y una segunda función para dividir en el conjunto de entrenamiento y en el conjunto de validación. Para esto se usará un escala de 75:25

In [11]:
# Función para dividir el objetivo de las características
def features_target(data):
    features = data.drop('Price', axis=1)
    target = data['Price']
    return features, target

In [12]:
# Función para dividir en los conjuntos de validación y entrenamiento
def train_valid(features, target):
    fatures_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25,
                                                                                 random_state=12345)
    return fatures_train, features_valid, target_train, target_valid

### Regresión Linear (Para prueba de cordura)

La regresión linear no es muy buena para el ajuste de hiperparámetros, por lo que podemos usarlo para una prueba de cordura. Esto significa que podemos usar el valor de RMSE como punto de partida, si los modelos siguientes tienen un valor de RMSE mayor al de la regresión lineal, quiere decir que el modelo no fue bueno.

En este modelo vamos a utilizar los datos con codificación OHE.

In [13]:
# Dividimos en features y target
features_ohe, target_ohe = features_target(df_ohe)

In [14]:
# Set de entrenamiento y validación
fatures_train_ohe, features_valid_ohe, target_train_ohe, target_valid_ohe = train_valid(features_ohe, target_ohe)

Entrenamos el modelo de regresión

In [15]:
model_regresion = LinearRegression()
model_regresion.fit(fatures_train_ohe, target_train_ohe)

LinearRegression()

In [16]:
predict_valid_regression = model_regresion.predict(features_valid_ohe)

In [17]:
rmse_regression = mean_squared_error(target_valid_ohe, predict_valid_regression, squared=False)
print(rmse_regression)

3179.689416691931


***Conclusión***

Nuestro modelo, en promedio, falló por aproximadamente 3179 euros. Este es un valor de RMSE alto, sin embargo para ser tomado para nuestra prueba de cordura es aceptable.

### Bosque aleatorio de regresión

Para este moedelo utilizaremos el dataset con sodificación OHE.

Para este modelo se van a utilizar un ciclo for para modificar los hiperparámetros `n_estimators` y `depth` y ver cuál da como resultado un mejor RMSE.

In [18]:
# Se establece un contador para saber cuál es el mejor valor de  n_estimators y max_depth
best_rmse = 10000
best_est = 0
best_depth = 0

In [19]:
%%time
# Modelo en un ciclo for
for est in range(10, 11):
    for depth in range(1,11):
        model_tree = RandomForestRegressor(random_state=12345, n_estimators=est, max_depth=depth)
        model_tree.fit(fatures_train_ohe, target_train_ohe)
        predict_valid_tree = model_tree.predict(features_valid_ohe)
        rmse = mean_squared_error(target_valid_ohe, predict_valid_tree, squared=False)
        if rmse < best_rmse:
            best_rmse = rmse
            best_est = est
            best_depth = depth

print(f'El RMSE del mejor modelo, con depth = {best_depth} y n_estimators = {best_est}, es: {best_rmse}')

El RMSE del mejor modelo, con depth = 10 y n_estimators = 10, es: 2050.4500419873884
CPU times: user 4min, sys: 1.87 s, total: 4min 2s
Wall time: 4min 2s


***Conclusión***

Este modelo dice que, en promedio, las predicciones fallaron por aproximadamente 2050 euros, el cúal es menor a la prueba de cordura. Si alguno de los modelos siguientes tiene un RMSE menor a este, quiere decir que funcionan mejor que este modelo.

### CatBoostRegressor 

Utilizaremos la librería CatBoost para entrenar un modelo. Para este no es necesario usar datos con codificación OHE, por lo que se usarán los datos sin esta codificación.

In [20]:
# División del dataset en features y target usando la función features_target()
features, target = features_target(df)

In [21]:
# Conjuntos de entrenamiento y validación usando la función train_valid()
features_train, features_valid, target_train, target_valid = train_valid(features, target)

Para este modelo necesitamos pasar los nombres de las 6 columnas con características categóricas

In [22]:
cat_features = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']

Se entrena el modelo

In [23]:
%%time
model_cat = CatBoostRegressor(loss_function='RMSE', learning_rate=0.5, depth=4, n_estimators=500, random_state=12345)
model_cat.fit(features_train, target_train, cat_features=cat_features, verbose=100)

0:	learn: 3407.6615937	total: 504ms	remaining: 4m 11s
100:	learn: 1848.7613885	total: 35.1s	remaining: 2m 18s
200:	learn: 1792.8490559	total: 1m 9s	remaining: 1m 42s
300:	learn: 1759.8330726	total: 1m 43s	remaining: 1m 8s
400:	learn: 1735.8756541	total: 2m 18s	remaining: 34.2s
499:	learn: 1716.0910925	total: 2m 52s	remaining: 0us
CPU times: user 2min 53s, sys: 350 ms, total: 2min 54s
Wall time: 2min 54s


In [24]:
predict_valid_cat = model_cat.predict(features_valid)

In [25]:
rmse_cat = mean_squared_error(target_valid, predict_valid_cat, squared=False)
print(rmse_cat)

1786.2523826263018


***Conclusión***

Se probaron diferentes hiperparámetros para el entrenamiento. Por cuestiones de ahorro de tiempo, no se coloca el modelo dentro de ciclos for para ir variando algunos hiperparámetros.

Se encotró que con los hiperparámetros `learning_rate=0.5`, `depth=4`, `n_estimators=500` hay un mejor valor de RMSE, el cuál es de 1786.25. Pero el tiempo de entrenamiento es aproximadamente de **3 minutos y medio a 4 minutos y medio** en mi computadora.

### LightGBM

Se va a utilizar la librería LGBMRegressor para entrenar un modelo y obtener un RMSE. De igual manera que con CatBoost, no es neceario usar una codificación OHE.

Para esto vamos a utilizar los `features_train`, `features_valid`, `target_train`, `target_valid` que ya obtuvimos en el modelo anterior.

Para este modelo vamos a utilizar los índices de las columnas categóricas en lugar del nombre.

In [26]:
# Colocamos los índices de las columnas categóricas en una variable
categorical_indices = [features_train.columns.get_loc(col) for col in cat_features]
print(categorical_indices)

[0, 2, 4, 7, 8, 9]


**Nota del estudiante**

Al pasar las columnas categóricas como tipo `object` el modelo presentaba errores, por lo que se cambió el tipo de datos a `category`, de esta forma el modelo trabajó sin problemas

In [27]:
#Convierte el tipo de datos de a categóricos
features_train = features_train.astype({'VehicleType': 'category', 'Gearbox': 'category', 'Model': 'category', 'FuelType': 'category', 'Brand': 'category', 'NotRepaired': 'category'})
features_valid = features_valid.astype({'VehicleType': 'category', 'Gearbox': 'category', 'Model': 'category', 'FuelType': 'category', 'Brand': 'category', 'NotRepaired': 'category'})

In [28]:
%%time
# Se entrena el modelo
modelo_light = LGBMRegressor(learning_rate=0.5, max_depth=4, n_estimators=500, random_state=12345, categorical_feature= categorical_indices)
modelo_light.fit(features_train, target_train, verbose=100)

/opt/conda/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.9/site-packages/lightgbm/basic.py:1487: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'
/opt/conda/lib/python3.9/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


CPU times: user 13.1 s, sys: 40.5 ms, total: 13.1 s
Wall time: 13.2 s


LGBMRegressor(categorical_feature=[0, 2, 4, 7, 8, 9], learning_rate=0.5,
              max_depth=4, n_estimators=500, random_state=12345)

In [29]:
predict_valid_light = modelo_light.predict(features_valid)

In [30]:
rmse_light = mean_squared_error(target_valid, predict_valid_light, squared=False)
print(rmse_light)

1755.915320892312


***Conclusión***

Este modelo presentó un mejor valor de RMSE, el cuál es de 1755.91. Además el tiempo para entrenar el modelo es mucho menor.

### XGBoost 

Para esta librería es necesario usar los datos con codifiación OHE, ya que el modelo no puede hacer la distinción entre características categóricas y numéricas como los 2 modelos anteriores.


In [31]:
%%time
model_xgboost = XGBRegressor(learning_rate=0.5, n_estimators=500, max_depth=4, objective='reg:squarederror', random_state=12345)
model_xgboost.fit(fatures_train_ohe, target_train_ohe)

CPU times: user 29min 41s, sys: 1.81 s, total: 29min 43s
Wall time: 29min 44s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.5, max_delta_step=0,
             max_depth=4, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=500, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=12345,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [32]:
predict_valid_xgboost = model_xgboost.predict(features_valid_ohe)

In [33]:
rmse_xgboost = mean_squared_error(target_valid_ohe, predict_valid_xgboost, squared=False)
print(rmse_light)

1755.915320892312


***Conclusion***

El modelo obtuvo un RMSE de 1755.91, es igual que el modelo de LightGBM, sin embargo el modelo tardó mucho más en entrenar el modelo

## Análisis del modelo

Todos los modelos hechos pasan la prueba cordura en cuanto al RMSE. Además a cada modelo se le tomó el tiempo usando el magic command `%%time` para saber cuánto tardan en entrenar, con los hiperparámetros dados.

Estos son los resultados:

* Bosque aleatorio de regresión:
    1. RMSE = 2050.45
    2. Tiempo = 4 min
* CatBoost:
    1. RMSE = 1786.25
    2. Tiempo = 2 min 54 seg
* LightGBM
    1. RMSE = 1755.91
    2. Tiempo = 13.2 s
* XGBoost
    1. RMSE = 1755.91
    2. Tiempo = 29 min 44s
    
Estos es el tiempo que tardo mi computadora en realizar los entrenamientos, pero con una computadora diferente puede variar el tiempo.

***Conclusión***

Los modelos que usan potenciación del gradiente dan mejor resultado, en cuanto al RMSE, al manejar diferentes hiperparámetros.

El modelo con la librería **LightGBM** dio el mejor resultado de RMSE y el menor tiempo de ejecución. Por lo que es el mejor modelo para realizar la tarea de predicción del precio de autos.

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [x]  El código no tiene errores- [ ]  Las celdas con el código han sido colocadas en orden de ejecución- [ ]  Los datos han sido descargados y preparados- [ ]  Los modelos han sido entrenados
- [x]  Se realizó el análisis de velocidad y calidad de los modelos